### Load packages

In [64]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
%matplotlib inline
from sklearn import cross_validation as cv
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn import preprocessing
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model

### Load csv

In [106]:
motionloc = 'C:/Users/Valued Customer/Desktop/motion_feature.csv'
nomotionloc = 'C:/Users/Valued Customer/Desktop/nomotion_feature.csv'
motion = pd.read_csv(motionloc, header=None,skip_blank_lines=True) # load whole table, row 22 is NA
nomotion = pd.read_csv(nomotionloc, header=None,skip_blank_lines=True)
motion_data = motion.ix[1:,2:]  # column 2-end are features, row 0 is column name
nomotion_data = nomotion.ix[1:,2:]
motion_class = np.empty(len(motion)-1) 
motion_class [:] = 1 #
#motion.ix[1:,7] # column 7 is class
nomotion_class = np.empty(len(nomotion)-1) 
nomotion_class [:] = 0 
#nomotion.ix[1:,7]
x=motion_data[36:].append(nomotion_data[51:],ignore_index = True)
x=x.astype('float64') # convert string to float
y=pd.concat([pd.DataFrame(motion_class[36:]),pd.DataFrame(nomotion_class[51:])],ignore_index = True)
for i in range(len(y)):
    y.ix[i]=y.ix[i].astype('category') # convert string to categorical

### Exploratory analysis

In [3]:
# data_all=x
# data_all['class']=y

In [4]:
# _=data_all.boxplot(by='class')

In [5]:
# _=data_all.groupby('class').boxplot() 

In [6]:
# _=scatter_matrix(x, alpha=0.2, figsize=(6, 6))#, diagonal='kde')

In [7]:
# _=data_all.hist()

### Split training and testing

In [107]:
x_train, x_test, y_train, y_test=cv.train_test_split(x,y,test_size=0.1)

### Data preprocessing

In [108]:
x_train_scaled = preprocessing.scale(x_train)
scaler = preprocessing.StandardScaler().fit(x_train)
x_test_scaled= scaler.transform(x_test)

In [109]:
data_scaled=pd.DataFrame(x_train_scaled)
data_scaled['class']=y_train

In [16]:
# data_all=x
# data_all['class']=y
# _=data_all.boxplot(by='class')
# _=data_all.groupby('class').boxplot() 
# _=scatter_matrix(x, alpha=0.2, figsize=(6, 6))#, diagonal='kde')
# _=data_all.hist()

### Build classifier

In [110]:
names=['LDA','QDA','logistic regression','random forest','SVC','KNN']
classifiers=[LinearDiscriminantAnalysis(),
             QuadraticDiscriminantAnalysis(),
             LogisticRegression(),
             RandomForestClassifier(max_depth=5, n_estimators=3, max_features=1),
             SVC(gamma=2, C=1),
             KNeighborsClassifier(3)]
for name, clf in zip(names,classifiers):
    #clf.fit(x_train_scaled,y_train)
    #print(clf.score(x_test_scaled,y_test))
    score = cross_validation.cross_val_score(clf, x_train_scaled, y_train[0], cv=5)#,scoring=scoring)
    print'{}: accuracy {}+/-{}'.format(name,score.mean(),score.std())

LDA: accuracy 0.740740740741+/-0.0560618901583
QDA: accuracy 0.643555555556+/-0.102488541967
logistic regression: accuracy 0.797333333333+/-0.094807407118
random forest: accuracy 0.780148148148+/-0.0436833599857
SVC: accuracy 0.796148148148+/-0.0478277659828
KNN: accuracy 0.787555555556+/-0.0392925086316


### Build Classifier with first 5 of the features

In [111]:
for name, clf in zip(names,classifiers):
    #clf.fit(x_train_scaled,y_train)
    #print(clf.score(x_test_scaled,y_test))
    score = cross_validation.cross_val_score(clf, x_train_scaled[:,0:5], y_train[0], cv=5)#,scoring=scoring)
    print'{}: accuracy {}+/-{}'.format(name,score.mean(),score.std())

LDA: accuracy 0.709333333333+/-0.0416546649382
QDA: accuracy 0.596740740741+/-0.0752347583332
logistic regression: accuracy 0.811555555556+/-0.0797824201682
random forest: accuracy 0.764740740741+/-0.0434899985349
SVC: accuracy 0.796148148148+/-0.0478277659828
KNN: accuracy 0.764148148148+/-0.0479267845822


### Lasso

In [72]:
lasso = linear_model.Lasso()
alphas = np.logspace(-4, -.5, 10)
scores=[]
scores_std=[]
for alpha in alphas:
    lasso.alpha = alpha
    this_scores = cross_validation.cross_val_score(lasso, x_train_scaled, y_train[0], n_jobs=1)
    scores.append(np.mean(this_scores))
    scores_std.append(np.std(this_scores))

In [73]:
scores

[0.13345508391063854,
 0.13849516791776914,
 0.14066612909843335,
 0.14326339509377842,
 0.14516793688692206,
 0.14885596016596433,
 0.15344222115238385,
 0.13841959843946838,
 0.10203989472908943,
 -0.012189163180254647]

### Ridge

In [74]:
n_alphas = 10
alphas = np.logspace(-10, -1, n_alphas)
clf = linear_model.Ridge(fit_intercept=False)
scores =[]
for a in alphas:
    clf.set_params(alpha=a)
    this_scores = cross_validation.cross_val_score(clf, x_train_scaled, y_train[0], cv=3)#,scoring=scoring)
    scores.append(np.mean(this_scores))

In [75]:
scores

[-1.5435373642480001,
 -1.5435373667350472,
 -1.5435373916053834,
 -1.543537640306015,
 -1.5435401270243145,
 -1.543564965339999,
 -1.5438104106298891,
 -1.5459585595639325,
 -1.5534544050177692,
 -1.5540032378555277]